# Extracting Keywords Script

In [4]:
news_data = []

In [5]:
# code for extracting news data using mediastack api. Data has already been extracted and is saved in the newsarticle.json file
# Thus, code is commented out

# import http.client, urllib.parse, json

# conn = http.client.HTTPConnection('api.mediastack.com')

# for i in range(0, 6):
#     params = urllib.parse.urlencode({
#         'access_key': '38f726aef9cabcd89940e27f93c99e3f',
#         'categories': 'politics',
#         'countries': 'us',
#         'sort': 'published_desc',
#         'limit': 100,
#         'offset': i*100,
#         # 'keywords': 'election',
#         'date': '2024-10-15,2024-11-26'
#     })

#     conn.request('GET', '/v1/news?{}'.format(params))
    
#     res = conn.getresponse()
#     data = res.read().decode('utf-8')
#     news_data += list(json.loads(data)["data"])

In [127]:
def read_file(file_name):
    with open(file_name, "r") as json_file:
        data = json.load(json_file)
    return data


def write_file(data, file_name):
    with open(file_name, "w") as json_file:
        json.dump(data, json_file, indent=4)  # `indent=4` makes it pretty-printed

news_data = read_file("newsarticles.json")

282


In [128]:
# find cosine similarity of relevant categories to the election

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Sample documents
documents = news_data

# Create a count matrix
vectorizer = CountVectorizer()
count_matrix = vectorizer.fit_transform(documents).toarray()

# Create a word-to-index mapping
vocab = vectorizer.get_feature_names_out()
word_to_index = {word: idx for idx, word in enumerate(vocab)}

# Find co-occurrence vectors for a target word
keywords = {"economy": [], "democracy": [], "security": [], "immigration": [], "education": [], "healthcare": [], "abortion": []}
for keyword in keywords:
    if keyword in word_to_index:
        target_idx = word_to_index[keyword]
        target_vector = count_matrix[:, target_idx]
    
        # Calculate cosine similarity with all other words
        similarities = cosine_similarity(target_vector.reshape(1, -1), count_matrix.T).flatten()
    
        # Get top related words
        related_indices = similarities.argsort()[::-1][1:21]
        print(f"Words most related to '{keyword}':")
        for idx in related_indices:
            keywords[keyword].append(vocab[idx])
        print(keywords[keyword])
    else:
        print(f"'{target_word}' not found in the vocabulary.")

Words most related to 'economy':
['prices', 'inflation', 'triggering', 'stock', 'scores', 'expensive', 'felt', 'rates', 'respondents', 'professional', 'economic', 'stamps', 'repercussions', 'roxanne', 'afloat', 'aquaculture', 'priced', 'manageable', 'succumbing', 'necessities']
Words most related to 'democracy':
['siphoned', 'mudde', 'legalized', 'hence', 'capitalism', 'rectify', 'columns', 'newest', 'reich', 'acquisitions', 'robertreich', 'stalled', 'nausea', 'misallocation', 'inbox', 'rigs', 'cas', 'eldercare', 'summoned', 'corporations']
Words most related to 'security':
['social', 'shortfall', 'finances', 'insolvency', '2031', 'footing', 'mismanage', 'proposing', 'waving', 'trustees', 'insolvent', 'modernize', 'holland', 'ss', '2033', 'breck', 'dumas', 'angrily', 'buckle', 'oasi']
Words most related to 'immigration':
['immigrants', 'policies', 'border', 'forgiveness', 'country', 'about', 'sues', 'migrants', 'illegally', 'wanted', 'harris', 'illegal', 'said', 'did', 'from', 'trump',

In [130]:
# save to file
write_file(keywords, "keywords.json")

# End of Script